In [6]:
import cvxpy as cp
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

# Конвертируем тенге в доллары (примерный курс)
KZT_to_USD = 0.0023
total_investment_KZT = 1000000  # Общая сумма инвестиций в тенге
total_investment = total_investment_KZT * KZT_to_USD  # Конвертация в доллары

# Загрузим исторические данные для акций и криптовалют
tickers = ['AAPL', 'MSFT', 'GOOGL']
crypto_tickers = ['BTC-USD', 'ETH-USD']

data_stocks = yf.download(tickers, start='2020-01-01', end=datetime.now())['Adj Close']
data_crypto = yf.download(crypto_tickers, start='2020-01-01', end=datetime.now())['Adj Close']

returns_stocks = data_stocks.pct_change().dropna()
returns_crypto = data_crypto.pct_change().dropna()

mean_returns_stocks = returns_stocks.mean().values  # Приведем к numpy массиву
mean_returns_crypto = returns_crypto.mean().values  # Приведем к numpy массиву

cov_matrix_stocks = returns_stocks.cov().values  # Ковариационная матрица для акций
cov_matrix_crypto = returns_crypto.cov().values  # Ковариационная матрица для криптовалют

num_assets_stocks = len(tickers)
num_assets_crypto = len(crypto_tickers)

# Определите переменные
shares_stocks = cp.Variable(num_assets_stocks, integer=True)
shares_crypto = cp.Variable(num_assets_crypto)

# Целевая функция - максимизация ожидаемой прибыли
objective = cp.Maximize(mean_returns_stocks @ shares_stocks + mean_returns_crypto @ shares_crypto)

# Ограничения
risk_tolerance = 0.02  # Примерное допустимое значение стандартного отклонения (2%)

constraints = [
    shares_stocks >= 0,  # Минимальное количество акций может быть 0
    shares_crypto >= 0,  # Минимальное количество криптовалют может быть 0
    shares_stocks @ data_stocks.iloc[-1].values + shares_crypto @ data_crypto.iloc[-1].values <= total_investment,  # Общая стоимость портфеля
    cp.quad_form(shares_stocks, cov_matrix_stocks) + cp.quad_form(shares_crypto, cov_matrix_crypto) <= risk_tolerance  # Ограничение на риск
]

# Оптимизация
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)

# Получим текущие цены акций и криптовалют
current_prices_stocks = data_stocks.iloc[-1].values
current_prices_crypto = data_crypto.iloc[-1].values

investment_amounts_stocks = shares_stocks.value * current_prices_stocks
investment_amounts_crypto = shares_crypto.value * current_prices_crypto

# Вывод результатов
print("Оптимальное количество акций и криптовалют, и суммы инвестиций:")
for ticker, num_shares, price, investment in zip(tickers, shares_stocks.value, current_prices_stocks, investment_amounts_stocks):
    if num_shares > 0:
        investment_KZT = investment / KZT_to_USD  # Конвертация в тенге
        investment_percentage = (investment / total_investment) * 100  # Процент от общей суммы
        print(f"{ticker}: {int(num_shares)} акций, сумма инвестирования: ${investment:.2f} / {investment_KZT:.2f} тенге ({investment_percentage:.2f}%)")

for ticker, num_shares, price, investment in zip(crypto_tickers, shares_crypto.value, current_prices_crypto, investment_amounts_crypto):
    if num_shares > 0:
        investment_KZT = investment / KZT_to_USD  # Конвертация в тенге
        investment_percentage = (investment / total_investment) * 100  # Процент от общей суммы
        print(f"{ticker}: {num_shares:.4f} криптовалюты, сумма инвестирования: ${investment:.2f} / {investment_KZT:.2f} тенге ({investment_percentage:.2f}%)")

# Общая сумма инвестиций в долларах и тенге
total_investment_dollars = np.sum(investment_amounts_stocks) + np.sum(investment_amounts_crypto)
total_investment_tenge = total_investment_dollars / KZT_to_USD

print(f"\nОбщая сумма инвестиций: ${total_investment_dollars:.2f} / {total_investment_tenge:.2f} тенге")

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  2 of 2 completed

Оптимальное количество акций и криптовалют, и суммы инвестиций:
AAPL: 6 акций, сумма инвестирования: $1345.86 / 585156.52 тенге (58.52%)
MSFT: 1 акций, сумма инвестирования: $177.66 / 77243.48 тенге (7.72%)
ETH-USD: 0.2230 криптовалюты, сумма инвестирования: $776.48 / 337600.00 тенге (33.76%)

Общая сумма инвестиций: $2300.00 / 1000000.00 тенге


In [14]:
def kelly_criterion(probability, odds):
    """
    Рассчитывает размер ставки по критерию Келли.
    :param probability: Вероятность победы (от 0 до 1)
    :param odds: Коэффициент ставки
    :return: Оптимальный процент капитала для ставки
    """
    return (probability * (odds - 1) - (1 - probability)) / (odds - 1)

def calculate_bets(bankroll, events, kelly_fraction=0.5, max_bet_fraction=0.05, min_bet_amount=10, min_events=3):
    """
    Рассчитывает суммы ставок на основе заданного банка и событий с уменьшением рисков.
    :param bankroll: Общий банк
    :param events: Список событий с вероятностями и коэффициентами
    :param kelly_fraction: Доля ставки по критерию Келли (например, 0.5 для половинной ставки)
    :param max_bet_fraction: Максимальная доля банка, которую можно поставить на одно событие
    :param min_bet_amount: Минимальная сумма ставки
    :param min_events: Минимальное количество событий для диверсификации
    :return: Словарь с рекомендованными ставками
    """
    bets = {}
    num_events = len(events)
    
    if num_events < min_events:
        print(f"Недостаточно событий для диверсификации. Необходимо минимум {min_events}.")
        return bets

    for event in events:
        probability = event['probability']
        odds_1 = event['odds_1']
        odds_2 = event['odds_2']
        
        # Проверка на адекватность коэффициентов и вероятностей
        if probability <= 0 or probability >= 1 or odds_1 <= 1 or odds_2 <= 1:
            print(f"Неверные данные для события {event['name']}")
            continue
        
        kelly_fraction_raw = kelly_criterion(probability, odds_1)
        kelly_fraction_adjusted = kelly_fraction * kelly_fraction_raw
        bet_amount_1 = bankroll * kelly_fraction_adjusted

        kelly_fraction_raw = kelly_criterion(1 - probability, odds_2)
        kelly_fraction_adjusted = kelly_fraction * kelly_fraction_raw
        bet_amount_2 = bankroll * kelly_fraction_adjusted
        
        # Ограничение на максимальную ставку
        max_bet = bankroll * max_bet_fraction
        bet_amount_1 = min(bet_amount_1, max_bet)
        bet_amount_2 = min(bet_amount_2, max_bet)
        
        # Убедиться, что ставка не превышает банк и больше или равна минимальной сумме ставки
        bet_amount_1 = max(0, min(bet_amount_1, bankroll))
        bet_amount_2 = max(0, min(bet_amount_2, bankroll))
        
        if bet_amount_1 >= min_bet_amount:
            bets[event['name']] = {'team_1': bet_amount_1}
        else:
            bets[event['name']] = {'team_1': 0}
        
        if bet_amount_2 >= min_bet_amount:
            bets[event['name']]['team_2'] = bet_amount_2
        else:
            bets[event['name']]['team_2'] = 0

    # Перераспределение банка, чтобы каждая ставка была не больше доступного банка
    total_bet = sum(bet['team_1'] + bet['team_2'] for bet in bets.values())
    if total_bet > bankroll:
        for event in bets:
            bets[event]['team_1'] = bets[event]['team_1'] * (bankroll / total_bet)
            bets[event]['team_2'] = bets[event]['team_2'] * (bankroll / total_bet)
    
    return bets

# Пример использования
bankroll = 1000  # Общий банк для ставок

# Список событий с именами, вероятностями и коэффициентами
events = [
    {'name': 'Event 1', 'probability': 0.5, 'odds_1': 2.0, 'odds_2': 1.8},
    {'name': 'Event 2', 'probability': 0.4, 'odds_1': 2.5, 'odds_2': 1.6},
    {'name': 'Event 3', 'probability': 0.7, 'odds_1': 1.5, 'odds_2': 2.2},
    {'name': 'Event 4', 'probability': 0.6, 'odds_1': 1.8, 'odds_2': 2.0},
    {'name': 'Event 5', 'probability': 0.55, 'odds_1': 1.9, 'odds_2': 2.1},
]

bets = calculate_bets(bankroll, events, kelly_fraction=0.5, max_bet_fraction=0.05, min_bet_amount=10, min_events=3)

# Вывод результатов
for event_name, bet in bets.items():
    if bet['team_1'] > 0:
        print(f"{event_name}: Ставка на команду 1 - {bet['team_1']:.2f}")
    if bet['team_2'] > 0:
        print(f"{event_name}: Ставка на команду 2 - {bet['team_2']:.2f}")

Event 3: Ставка на команду 1 - 50.00
Event 4: Ставка на команду 1 - 50.00
Event 5: Ставка на команду 1 - 25.00


In [23]:
import yfinance as yf

def get_current_price(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    current_price = ticker.info["preMarketPrice"]
    return current_price

# Пример использования
ticker_symbol = "AAPL"
current_price = get_current_price(ticker_symbol)
print(f"Текущая цена {ticker_symbol}: {current_price}$")

KeyError: 'preMarketPrice'

In [21]:
current_price

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']